In [1]:
# import relevant modules and set options

import numpy as np
import pandas as pd
import os, sys

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# read in the data

#os.chdir('/Users/...')
used_cars = pd.read_csv("./used_cars_data.csv")

/var/folders/7q/kc_brkw94wl_0nrnfg39zqmw0000gn/T/ipykernel_12043/2812704651.py:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  used_cars = pd.read_csv("used_cars_data.csv")


In [3]:
# remove features with NA% >= 0.48
cnt_percentage = []
for col in used_cars.columns:
    cnt_percentage.append((col, used_cars[col].count() / used_cars.shape[0]))
    
cnt_percentage.sort(key = lambda x: x[1])
# cnt_percentage

selected_feature = []
for item in cnt_percentage:
    if item[1] >= 0.48:
        selected_feature.append(item[0])

used_cars = used_cars[selected_feature] # subset to remove features with greater than 48% NA

In [4]:
cnt_percentage

[('combine_fuel_economy', 0.0),
 ('is_certified', 0.0),
 ('vehicle_damage_category', 0.0),
 ('bed', 0.006522579698937348),
 ('cabin', 0.021177384301542646),
 ('is_oemcpo', 0.045120065065799124),
 ('is_cpo', 0.060965187130838254),
 ('bed_height', 0.14303075958987213),
 ('bed_length', 0.14303075958987213),
 ('owner_count', 0.4943357421901041),
 ('fleet', 0.5244746736710177),
 ('frame_damaged', 0.5244746736710177),
 ('has_accidents', 0.5244746736710177),
 ('isCab', 0.5244746736710177),
 ('salvage', 0.5244746736710177),
 ('theft_title', 0.5244746736710177),
 ('franchise_make', 0.8091242116771776),
 ('torque', 0.8274046346048719),
 ('city_fuel_economy', 0.8362405167931094),
 ('highway_fuel_economy', 0.8362405167931094),
 ('power', 0.8395268063092493),
 ('interior_color', 0.8720063732483567),
 ('main_picture_url', 0.8769706403914614),
 ('major_options', 0.933318222423701),
 ('engine_displacement', 0.9425387661497847),
 ('horsepower', 0.9425387661497847),
 ('back_legroom', 0.9469110411861175)

In [5]:
# remove rows where more than 80% of observations are NA
print(used_cars.shape)
used_cars.dropna(axis=0, thresh= int(0.8 * used_cars.shape[1]), inplace=True)

print(used_cars.shape) # removes less than 200,000 rows

(3000040, 57)
(2815307, 57)


In [6]:
# only keep used cars listed in the last three months
used_cars = used_cars[used_cars['listed_date'] > '2020-06-13'] 

# drop used cars older than 10 years
used_cars = used_cars[used_cars['year'] >= 2010]

print(used_cars.shape) # (2057964, 57)

(2057964, 57)


In [7]:
# clean features in the data
# convert to appropriate types
# drop unncessary columns

def combine_categories_to_other(freq_threshold, col):
        value_count_df = used_cars[col].value_counts().rename_axis('unique_values').reset_index(name='counts')
        other = value_count_df.loc[value_count_df['counts'] <= freq_threshold, 'unique_values'].to_list()
        used_cars.loc[used_cars[col].isin(other), col] = 'Other'

#  convert numeric data with unit (in object format) to numeric
for var in ['back_legroom', 'front_legroom', 'fuel_tank_volume', 'height', 'length', 'maximum_seating', 'wheelbase', 'width']:
    used_cars[var] = used_cars[var].str.split(" ", expand=True)[0] # remove units (units are always the same within a variable)
    used_cars[var] = [None if var == '--' else var for var in used_cars[var]] # treat '--' as missing
    used_cars[var] = pd.to_numeric(used_cars[var])

# convert torque and power
used_cars['torque'] = used_cars['torque'].str.replace(',', '', regex = False)
used_cars['torque'] = used_cars['torque'].str.replace('[^0-9]', ' ', regex=True)
used_cars[['torque_1','torque_2']] = used_cars.torque.str.split(expand=True)
    
used_cars['power'] = used_cars['power'].str.replace(',', '', regex = False)
used_cars['power'] = used_cars['power'].str.replace('[^0-9]', ' ', regex=True)
used_cars[['power_1','power_2']] = used_cars.power.str.split(expand=True)

# convert dealer_zip to string; pad with leading 0s to have 5 digits
used_cars['dealer_zip'] = used_cars['dealer_zip'].apply(str)
used_cars['dealer_zip'] = used_cars['dealer_zip'].apply(lambda x: x.zfill(5))

# create simplified version of engine type
used_cars['engine_type_simple'] = used_cars['engine_type'].str.split(' ').str[0] # take characters before the space
# take just the engine shape
used_cars['engine_shape'] = used_cars['engine_type'].str[0] # first character is engine shape

# convert listed_date to datetime
used_cars['listed_date'] = pd.to_datetime(used_cars['listed_date'], yearfirst=True)

# drop features
manually_dropped_col = ['description','engine_cylinders', 'engine_type', 'exterior_color', 'fleet', 'interior_color', \
                        'isCab', 'main_picture_url', 'savings_amount', 'sp_id', 'sp_name', 'torque', 'transmission_display', \
                        'trimId', 'power', 'wheel_system_display']
    
for drop_col in manually_dropped_col:
    if drop_col in used_cars.columns:
        used_cars.drop(drop_col, inplace = True, axis = 1)


In [8]:
# check data types
sorted(used_cars.columns)

['back_legroom',
 'body_type',
 'city',
 'city_fuel_economy',
 'daysonmarket',
 'dealer_zip',
 'engine_displacement',
 'engine_shape',
 'engine_type_simple',
 'frame_damaged',
 'franchise_dealer',
 'franchise_make',
 'front_legroom',
 'fuel_tank_volume',
 'fuel_type',
 'has_accidents',
 'height',
 'highway_fuel_economy',
 'horsepower',
 'is_new',
 'latitude',
 'length',
 'listed_date',
 'listing_color',
 'listing_id',
 'longitude',
 'major_options',
 'make_name',
 'maximum_seating',
 'mileage',
 'model_name',
 'owner_count',
 'power_1',
 'power_2',
 'price',
 'salvage',
 'seller_rating',
 'theft_title',
 'torque_1',
 'torque_2',
 'transmission',
 'trim_name',
 'vin',
 'wheel_system',
 'wheelbase',
 'width',
 'year']

In [26]:
# summarize the data
used_cars.describe()

,owner_count,city_fuel_economy,highway_fuel_economy,engine_displacement,horsepower,back_legroom,front_legroom,fuel_tank_volume,height,length,maximum_seating,wheelbase,width,mileage,seller_rating,daysonmarket,latitude,listed_date,listing_id,longitude,price,year
count,1.139533e+06,1.819630e+06,1.819630e+06,2.054879e+06,2.054879e+06,2.009508e+06,2.047210e+06,2.056860e+06,2.056873e+06,2.056871e+06,2.056854e+06,2.056886e+06,2.056873e+06,1.968724e+06,2.034842e+06,2.057964e+06,2.057964e+06,2057964,2.057964e+06,2.057964e+06,2.057964e+06,2.057964e+06
mean,1.382729e+00,2.234595e+01,2.930000e+01,2.986358e+03,2.507557e+02,3.807919e+01,4.223878e+01,1.866308e+01,6.596489e+01,1.943847e+02,5.511638e+00,1.157158e+02,7.851076e+01,2.957717e+04,4.270068e+00,3.033803e+01,3.691610e+01,2020-08-11 04:13:42.829554432,2.790609e+08,-9.062040e+01,2.969671e+04,2.018155e+03
min,1.000000e+00,9.000000e+00,1.100000e+01,7.000000e+02,7.000000e+01,0.000000e+00,3.350000e+01,1.900000e+00,4.370000e+01,1.061000e+02,2.000000e+00,7.350000e+01,6.140000e+01,0.000000e+00,1.000000e+00,0.000000e+00,1.844390e+01,2020-06-14 00:00:00,2.739765e+08,-1.579130e+02,3.869000e+02,2.010000e+03
25%,1.000000e+00,1.800000e+01,2.500000e+01,2.000000e+03,1.760000e+02,3.610000e+01,4.100000e+01,1.480000e+01,5.810000e+01,1.821000e+02,5.000000e+00,1.063000e+02,7.250000e+01,6.000000e+00,4.000000e+00,1.200000e+01,3.346150e+01,2020-07-27 00:00:00,2.776748e+08,-9.709420e+01,1.881500e+04,2.017000e+03
50%,1.000000e+00,2.100000e+01,2.900000e+01,2.500000e+03,2.460000e+02,3.830000e+01,4.200000e+01,1.800000e+01,6.620000e+01,1.907000e+02,5.000000e+00,1.112000e+02,7.770000e+01,1.710200e+04,4.333333e+00,2.500000e+01,3.770610e+01,2020-08-16 00:00:00,2.794768e+08,-8.689360e+01,2.650500e+04,2.019000e+03
75%,2.000000e+00,2.600000e+01,3.300000e+01,3.600000e+03,3.030000e+02,3.990000e+01,4.310000e+01,2.110000e+01,7.060000e+01,2.012000e+02,6.000000e+00,1.190000e+02,8.280000e+01,4.289200e+04,4.600000e+00,4.500000e+01,4.092330e+01,2020-08-30 00:00:00,2.807565e+08,-8.047290e+01,3.767500e+04,2.020000e+03
max,1.100000e+01,1.270000e+02,1.080000e+02,8.400000e+03,9.490000e+02,5.310000e+01,6.700000e+01,6.400000e+01,1.176000e+02,2.900000e+02,1.500000e+01,2.010000e+02,1.090000e+02,4.290461e+06,5.000000e+00,9.500000e+01,6.117200e+01,2020-09-13 00:00:00,2.820222e+08,-6.607850e+01,2.749950e+06,2.021000e+03
std,7.027046e-01,6.312808e+00,6.230222e+00,1.343697e+03,8.988450e+01,3.086972e+00,1.529419e+00,5.390465e+00,7.531172e+00,2.004188e+01,1.158753e+00,1.454096e+01,7.582544e+00,3.851566e+04,5.027671e-01,2.272533e+01,4.996591e+00,NaN,2.017791e+06,1.391205e+01,1.684625e+04,2.533924e+00


In [9]:
# remove obvious outliers

# mileage
outliers = np.where(used_cars['mileage'] >= 999999, True, False) # mileage over 999999 is probably some data entry error
used_cars.loc[outliers, 'mileage'] = np.nan

# price
# find 99.9th percentile for car price
p99_9 = np.percentile(used_cars['price'], 99.9) # remove any observations above this percentile ($147,000)
outliers_price = np.where(used_cars['price'] > p99_9, True, False)
used_cars = used_cars[outliers_price == False]

used_cars.describe()


,owner_count,city_fuel_economy,highway_fuel_economy,engine_displacement,horsepower,back_legroom,front_legroom,fuel_tank_volume,height,length,maximum_seating,wheelbase,width,mileage,seller_rating,daysonmarket,latitude,listed_date,listing_id,longitude,price,year
count,1.138227e+06,1.818041e+06,1.818041e+06,2.052824e+06,2.052824e+06,2.008825e+06,2.046155e+06,2.054818e+06,2.054818e+06,2.054816e+06,2.054799e+06,2.054831e+06,2.054818e+06,1.966730e+06,2.032812e+06,2.055906e+06,2.055906e+06,2055906,2.055906e+06,2.055906e+06,2.055906e+06,2.055906e+06
mean,1.382435e+00,2.235288e+01,2.930774e+01,2.984668e+03,2.504248e+02,3.807893e+01,4.223907e+01,1.866052e+01,6.597492e+01,1.943900e+02,5.513645e+00,1.157210e+02,7.850732e+01,2.959620e+04,4.269767e+00,3.033223e+01,3.691824e+01,2020-08-11 04:21:54.256198400,2.790614e+08,-9.061868e+01,2.950156e+04,2.018155e+03
min,1.000000e+00,9.000000e+00,1.100000e+01,7.000000e+02,7.000000e+01,0.000000e+00,3.350000e+01,1.900000e+00,4.580000e+01,1.061000e+02,2.000000e+00,7.350000e+01,6.140000e+01,0.000000e+00,1.000000e+00,0.000000e+00,1.844390e+01,2020-06-14 00:00:00,2.739765e+08,-1.579130e+02,3.869000e+02,2.010000e+03
25%,1.000000e+00,1.800000e+01,2.500000e+01,2.000000e+03,1.760000e+02,3.610000e+01,4.100000e+01,1.480000e+01,5.810000e+01,1.821000e+02,5.000000e+00,1.063000e+02,7.250000e+01,6.000000e+00,4.000000e+00,1.200000e+01,3.346570e+01,2020-07-27 00:00:00,2.776760e+08,-9.709220e+01,1.880000e+04,2.017000e+03
50%,1.000000e+00,2.100000e+01,2.900000e+01,2.500000e+03,2.450000e+02,3.830000e+01,4.200000e+01,1.800000e+01,6.620000e+01,1.907000e+02,5.000000e+00,1.112000e+02,7.770000e+01,1.716000e+04,4.333333e+00,2.500000e+01,3.770890e+01,2020-08-16 00:00:00,2.794773e+08,-8.689400e+01,2.650000e+04,2.019000e+03
75%,2.000000e+00,2.600000e+01,3.300000e+01,3.600000e+03,3.020000e+02,3.990000e+01,4.310000e+01,2.110000e+01,7.060000e+01,2.012000e+02,6.000000e+00,1.190000e+02,8.280000e+01,4.292500e+04,4.600000e+00,4.500000e+01,4.092390e+01,2020-08-30 00:00:00,2.807567e+08,-8.047870e+01,3.761200e+04,2.020000e+03
max,1.100000e+01,1.270000e+02,1.080000e+02,8.400000e+03,8.080000e+02,4.750000e+01,6.700000e+01,6.400000e+01,1.176000e+02,2.900000e+02,1.500000e+01,2.010000e+02,1.090000e+02,6.318350e+05,5.000000e+00,9.500000e+01,6.117200e+01,2020-09-13 00:00:00,2.820222e+08,-6.607850e+01,1.477660e+05,2.021000e+03
std,7.021992e-01,6.310256e+00,6.226202e+00,1.342838e+03,8.927734e+01,3.086472e+00,1.528644e+00,5.391758e+00,7.521547e+00,2.004650e+01,1.156738e+00,1.454372e+01,7.584294e+00,3.836559e+04,5.027967e-01,2.272221e+01,4.995619e+00,NaN,2.017530e+06,1.390865e+01,1.507473e+04,2.534275e+00


In [28]:
# export the data
used_cars.to_csv("used_cars_cleaned.csv")

used_cars.shape

(2055906, 47)

Additional Considerations/To Dos

In [ ]:
# check for redundancies in "franchise_make": for example, Rover vs. Land Rover? Are they the same make?
# are make_name and franchise_make the same? should we keep just one?

In [ ]:
# do we need to convert major_options to separate columns for each option?